Control-Flow-Prediction: Comparison of Models
--

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
from exp.eventlog import *

In [ ]:
from exp.dl_utils import *

In [ ]:
from exp.control_flow_prediction import *

In [4]:
data_sets=[]
models=[]

processing_params={'seed': 42}
training_params={}
evaluation_params={}

In [ ]:
def train_model(model,train,valid):
    pass

def measure_next_step_prediction(model,test):
    pass

def measure_suffix_prediction(model,test):
    pass

In [5]:
def run(data_sets=data_sets,models=models,processing_params=processing_params,
         training_params=training_params,evaluation_params=evaluation_params):
    
    for data_set in progress_bar(data_sets):
        train,valid,test=process_data_set(data_set,processing_params)
        for model in progress_bar(models):
            res_df=train_model(model,train,valid)
            res_df=measure_next_step_prediction(model,test)
            res_df=measure_suffix_prediction(model,test)
        
    